In [1]:
import fitz 

def extract_text_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(document)):
        page = document[page_num]
        text += page.get_text()
    document.close()
    return text.strip()

pdf_text = extract_text_from_pdf("report.pdf")
print(pdf_text[:500])  # Preview first 500 characters


Hello
The name is Parakh
